In [ ]:
import tweepy
import re

In [ ]:
pip install tweepy nltk google-cloud-language python-telegram-bot


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
config=pd.read_csv('/content/drive/My Drive/config.csv')

In [ ]:
twitterApiKey = config['twitterApiKey'][0]
twitterApiSecret = config['twitterApiSecret'][0]
twitterApiAccessToken = config['twitterApiAccessToken'][0]
twitterApiAccessTokenSecret = config['twitterApiAccessTokenSecret'][0]

In [ ]:
def authenticate(twitterApiKey,twitterApiSecret,twitterApiAccessToken,twitterApiAccessTokenSecret):
  auth = tweepy.OAuthHandler(twitterApiKey, twitterApiSecret)
  auth.set_access_token(twitterApiAccessToken, twitterApiAccessTokenSecret)
  twitterApi = tweepy.API(auth, wait_on_rate_limit = True)
  return twitterApi

In [ ]:
api = authenticate(twitterApiKey,twitterApiSecret,twitterApiAccessToken,twitterApiAccessTokenSecret)

In [ ]:
def search_tweets(keyword, total_tweets):
    today_datetime = datetime.today().now()
    yesterday_datetime = today_datetime - timedelta(days=1)
    today_date = today_datetime.strftime('%Y-%m-%d')
    yesterday_date = yesterday_datetime.strftime('%Y-%m-%d')
    api = authenticate(twitterApiKey,twitterApiSecret,twitterApiAccessToken,twitterApiAccessTokenSecret)
    search_result = tweepy.Cursor(api.search,
                                  q=keyword,
                                  since=yesterday_date,
                                  result_type='recent',
                                  lang='en').items(total_tweets)
    return search_result

In [ ]:
import re
from nltk.tokenize import WordPunctTokenizer

from datetime import datetime, timedelta


In [ ]:
def clean_tweets(tweet):
    user_removed = re.sub(r'@[A-Za-z0-9]+','',tweet.decode('utf-8'))
    link_removed = re.sub('https?://[A-Za-z0-9./]+','',user_removed)
    number_removed = re.sub('[^a-zA-Z]', ' ', link_removed)
    lower_case_tweet= number_removed.lower()
    tok = WordPunctTokenizer()
    words = tok.tokenize(lower_case_tweet)
    clean_tweet = (' '.join(words)).strip()
    return clean_tweet

In [ ]:

from enum import Enum
import numpy as np
from textblob import TextBlob

In [ ]:
def get_sentiment_score(tweet):
    return TextBlob(tweet).sentiment.polarity


In [ ]:
def analyze_tweets(keyword, total_tweets):
    score = 0
    tweets = search_tweets(keyword, total_tweets)
    for tweet in tweets:
        cleaned_tweet = clean_tweets(tweet.text.encode('utf-8'))
        sentiment_score = get_sentiment_score(cleaned_tweet)
        score += sentiment_score
        print('Tweet: {}'.format(cleaned_tweet))
        print('Score: {}\n'.format(sentiment_score))
    final_score = round((score / float(total_tweets)),2)
    return final_score

In [ ]:
def send_the_result(update,context):
    keyword = update.message.text
    final_score = analyze_tweets(keyword, 50)
    if final_score <= -0.25:
        status = 'NEGATIVE ❌'
    elif final_score <= 0.25:
        status = 'NEUTRAL ?'
    else:
        status = 'POSITIVE ✅'



In [ ]:
def message_handler(update, context):
  x=analyze_tweets(update.message.text,50)
  if x < -0.0:
      status = 'NEGATIVE ❌'
  elif x > 0.0:
      status = 'POSITIVE ✅'
  else:
      status = 'NEUTRAL !'
  update.message.reply_text(text='The Review is '+ status +' and the score is '+ str(x))


In [ ]:
from telegram.ext import Updater, MessageHandler, Filters, CommandHandler

def main():
    updater = Updater('1465058888:AAEyVrIcCLyp32PYfT7cVu-pam6SNjGJYRs')
    dp = updater.dispatcher
    dp.add_handler(CommandHandler('start', send_the_result))
    dp.add_handler(MessageHandler(filters=Filters.all, callback=message_handler))

    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()

Tweet: rt need bo records call prabhas need charming guy call mahesh babu need acting skills call ntr need dancer call al
Score: 0.35

Tweet: rt need bo records call prabhas need charming guy call mahesh babu need acting skills call ntr need dancer call al
Score: 0.35

Tweet: well done to all our boys who helped hilton u s win the bdyl cup on tuesday night against lichfield cc congratula
Score: 0.8

Tweet: need bo records call prabhas need charming guy call need acting skills call nee
Score: 0.35

Tweet: rt need bo records call prabhas need charming guy call mahesh babu need acting skills call ntr need dancer call al
Score: 0.35

Tweet: rt need bo records call prabhas need charming guy call mahesh babu need acting skills call ntr need dancer call al
Score: 0.35

Tweet: rt need bo records call prabhas need charming guy call mahesh babu need acting skills call ntr need dancer call al
Score: 0.35

Tweet: rt need bo records call prabhas need charming guy call mahesh babu need acting skills